In [48]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
import pandas as pd
import chroma

In [49]:
# Download Data for MNIST
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
train_kwargs = {"batch_size": 64}
test_kwargs = {"batch_size":1000}
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)

# We split the training pool into training and holdback for later sampling
train_size = int(0.5 * len(dataset1))
sample_from_size = len(dataset1) - train_size
train_dataset, sample_from_dataset = torch.utils.data.random_split(dataset1, [train_size, sample_from_size], generator=torch.Generator().manual_seed(42))

train_loader = torch.utils.data.DataLoader(train_dataset, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [50]:
# Setup our CNN to train on MNIST
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

device = torch.device("cpu")
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

def attach_forward_hook(model, array):
    return model.register_forward_hook(
        lambda model, input, output: array.append(output.data.detach().tolist())
    )

def infer(model, device, data_loader, resource_uris, label_classes, inference_classes):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target, resource_uri in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # why are we calculating loss here?
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            for resource_uri, label_class, inference_class in zip(resource_uri, target.data.detach().tolist(), pred.data.detach().flatten().tolist()):
                resource_uris.append(resource_uri)
                label_classes.append(label_class)
                inference_classes.append(inference_class)

    test_loss /= len(data_loader.dataset)

    print(
        "\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(data_loader.dataset), 100.0 * correct / len(data_loader.dataset)
        )
    )


# We modify the MNIST dataset to expose some information about the source data
# to allow us to uniquely identify an input in a way that we can recover it later
class CustomDataset(datasets.MNIST):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        resource_uri = f"{'train' if self.train else 't10k'}-images-idx3-ubyte-{index}"
        return img, target, resource_uri

In [51]:
# Train and test our model
epochs = 5
for epoch in range(1, epochs + 1):

    # Train
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10== 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

    # Determine Loss on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/30000 (0%)]	Loss: 2.298323
Train Epoch: 1 [640/30000 (2%)]	Loss: 1.234064
Train Epoch: 1 [1280/30000 (4%)]	Loss: 0.720270
Train Epoch: 1 [1920/30000 (6%)]	Loss: 0.650934
Train Epoch: 1 [2560/30000 (9%)]	Loss: 0.529386
Train Epoch: 1 [3200/30000 (11%)]	Loss: 0.430856
Train Epoch: 1 [3840/30000 (13%)]	Loss: 0.261618
Train Epoch: 1 [4480/30000 (15%)]	Loss: 0.260083
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.213215
Train Epoch: 1 [5760/30000 (19%)]	Loss: 0.336469
Train Epoch: 1 [6400/30000 (21%)]	Loss: 0.263782
Train Epoch: 1 [7040/30000 (23%)]	Loss: 0.169521
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.185041
Train Epoch: 1 [8320/30000 (28%)]	Loss: 0.303260
Train Epoch: 1 [8960/30000 (30%)]	Loss: 0.574012
Train Epoch: 1 [9600/30000 (32%)]	Loss: 0.316354
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.125933
Train Epoch: 1 [10880/30000 (36%)]	Loss: 0.129972
Train Epoch: 1 [11520/30000 (38%)]	Loss: 0.083715
Train Epoch: 1 [12160/30000 (41%)]	Loss: 0.128034
Train Epoch: 1 [12800/300

In [52]:
# Load the pre-trained model
model = Net()
model.load_state_dict(torch.load("mnist_cnn.pt"))
model.eval()
model.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [53]:
# Run Inference on all data and generate embeddings
inference_kwargs = {"batch_size": 1000}

train_embeddings = []
train_resource_uris = []
train_label_classes = []
train_inference_classes = []

sample_from_embeddings = []
sample_from_resource_uris = []
sample_from_label_classes = []
sample_from_inference_classes = []

train_mnist_data = CustomDataset("../data", train=True, transform=transform, download=True)
train_dataset, sample_from_dataset = torch.utils.data.random_split(train_mnist_data, [train_size, sample_from_size], generator=torch.Generator().manual_seed(42))

# from train
data_loader = torch.utils.data.DataLoader(train_dataset, **inference_kwargs)
hook = attach_forward_hook(model.fc2, train_embeddings)
infer(model, device, data_loader, train_resource_uris, train_label_classes, train_inference_classes)
hook.remove()

# from sample_from
data_loader = torch.utils.data.DataLoader(sample_from_dataset, **inference_kwargs)
attach_forward_hook(model.fc2, sample_from_embeddings)
infer(model, device, data_loader, sample_from_resource_uris, sample_from_label_classes, sample_from_inference_classes)

# remove one dimension from embeddings
train_embeddings = [item for sublist in train_embeddings for item in sublist]
sample_from_embeddings = [item for sublist in sample_from_embeddings for item in sublist]



Average loss: 0.0145, Accuracy: 29863/30000 (100%)


Average loss: 0.0489, Accuracy: 29598/30000 (99%)



In [54]:
# Load data into Chroma
api = chroma.get_api()
api.reset()
api.set_model_space("mnist")

api.add(
    embedding= train_embeddings,
    input_uri= train_resource_uris,
    dataset= "train",
    inference_class= train_inference_classes,
    label_class= train_label_classes,
    model_space= "mnist"
)
api.add(
    embedding= sample_from_embeddings,
    input_uri= sample_from_resource_uris,
    dataset= "test",
    inference_class= sample_from_inference_classes,
    label_class= sample_from_label_classes,
    model_space= "mnist"
)

print(api.count(model_space="mnist"))
api.process(training_dataset_name="train", inference_dataset_name="test", model_space="mnist")



Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
60000
time to fetch 60000 embeddings:  0.2901458740234375
time to fetch 30000 embeddings:  0.09539914131164551
time to fetch 30000 embeddings:  0.09370589256286621


/Users/jeff/.pyenv/versions/3.9.2/lib/python3.9/site-packages/hdbscan/prediction.py:382: UserWarning: Clusterer does not have any defined clusters, new data will be automatically predicted as noise.
  warn('Clusterer does not have any defined clusters, new data'


True

In [55]:
# Create an index and run ANN (commented out)
# api.create_index()
# results = api.get_nearest_neighbors(sample_from_embeddings[0], n_results=5)


In [56]:
# Get results back from Chroma
results = api.get_results(dataset_name="test", n_results=15000)
sample_from_crhoma_subset = [x for x in sample_from_dataset if x[2] in [y for y in results]]

In [57]:
# Randomly sample 15k results
random_sample_from_dataset = torch.utils.data.Subset(sample_from_dataset, torch.randperm(len(sample_from_dataset))[:15000])

In [60]:
# Train from scratch on the original cut and the sampled results

# Create a dataloader which is a combination of the original cut and the sampled results
train_sampled_dataset = torch.utils.data.ConcatDataset([train_dataset, sample_from_crhoma_subset])
train_sampled_loader = torch.utils.data.DataLoader(train_sampled_dataset, **train_kwargs)

sampled_model = Net()
optimizer = optim.Adadelta(sampled_model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Train and test our model
epochs = 5
for epoch in range(1, epochs + 1):

    # Train
    sampled_model.train()
    # emumerate through the dataloader
    for batch_idx, (data, target, _) in enumerate(train_sampled_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = sampled_model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10== 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_sampled_loader.dataset),
                    100.0 * batch_idx / len(train_sampled_loader),
                    loss.item(),
                )
            )

    # Determine Loss on the test set
    sampled_model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = sampled_model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    scheduler.step()

torch.save(sampled_model.state_dict(), "mnist_cnn_sampled.pt")

Train Epoch: 1 [0/42457 (0%)]	Loss: 2.297848
Train Epoch: 1 [640/42457 (2%)]	Loss: 1.385174
Train Epoch: 1 [1280/42457 (3%)]	Loss: 0.900825
Train Epoch: 1 [1920/42457 (5%)]	Loss: 0.691831
Train Epoch: 1 [2560/42457 (6%)]	Loss: 0.564817
Train Epoch: 1 [3200/42457 (8%)]	Loss: 0.382816
Train Epoch: 1 [3840/42457 (9%)]	Loss: 0.320076
Train Epoch: 1 [4480/42457 (11%)]	Loss: 0.321582
Train Epoch: 1 [5120/42457 (12%)]	Loss: 0.175594
Train Epoch: 1 [5760/42457 (14%)]	Loss: 0.264852
Train Epoch: 1 [6400/42457 (15%)]	Loss: 0.154354
Train Epoch: 1 [7040/42457 (17%)]	Loss: 0.175850
Train Epoch: 1 [7680/42457 (18%)]	Loss: 0.184211
Train Epoch: 1 [8320/42457 (20%)]	Loss: 0.221012
Train Epoch: 1 [8960/42457 (21%)]	Loss: 0.362074
Train Epoch: 1 [9600/42457 (23%)]	Loss: 0.463728
Train Epoch: 1 [10240/42457 (24%)]	Loss: 0.203111
Train Epoch: 1 [10880/42457 (26%)]	Loss: 0.127977
Train Epoch: 1 [11520/42457 (27%)]	Loss: 0.162777
Train Epoch: 1 [12160/42457 (29%)]	Loss: 0.152189
Train Epoch: 1 [12800/42457

In [61]:
# Train from scratch on the original cut and the sampled results

# Create a dataloader which is a combination of the original cut and the sampled results
train_random_dataset = torch.utils.data.ConcatDataset([train_dataset, random_sample_from_dataset])
train_random_loader = torch.utils.data.DataLoader(train_random_dataset, **train_kwargs)

random_model = Net()
optimizer = optim.Adadelta(random_model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Train and test our model
epochs = 5
for epoch in range(1, epochs + 1):

    # Train
    random_model.train()
    # emumerate through the dataloader
    for batch_idx, (data, target, _) in enumerate(train_random_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = random_model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10== 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_random_loader.dataset),
                    100.0 * batch_idx / len(train_random_loader),
                    loss.item(),
                )
            )

    # Determine Loss on the test set
    random_model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = random_model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

    scheduler.step()

torch.save(random_model.state_dict(), "mnist_cnn_random.pt")

Train Epoch: 1 [0/45000 (0%)]	Loss: 2.307080
Train Epoch: 1 [640/45000 (1%)]	Loss: 1.711250
Train Epoch: 1 [1280/45000 (3%)]	Loss: 0.790761
Train Epoch: 1 [1920/45000 (4%)]	Loss: 0.837130
Train Epoch: 1 [2560/45000 (6%)]	Loss: 0.409986
Train Epoch: 1 [3200/45000 (7%)]	Loss: 0.376750
Train Epoch: 1 [3840/45000 (9%)]	Loss: 0.215362
Train Epoch: 1 [4480/45000 (10%)]	Loss: 0.454628
Train Epoch: 1 [5120/45000 (11%)]	Loss: 0.182414
Train Epoch: 1 [5760/45000 (13%)]	Loss: 0.224168
Train Epoch: 1 [6400/45000 (14%)]	Loss: 0.178480
Train Epoch: 1 [7040/45000 (16%)]	Loss: 0.217367
Train Epoch: 1 [7680/45000 (17%)]	Loss: 0.258223
Train Epoch: 1 [8320/45000 (18%)]	Loss: 0.229271
Train Epoch: 1 [8960/45000 (20%)]	Loss: 0.604636
Train Epoch: 1 [9600/45000 (21%)]	Loss: 0.315153
Train Epoch: 1 [10240/45000 (23%)]	Loss: 0.187763
Train Epoch: 1 [10880/45000 (24%)]	Loss: 0.211865
Train Epoch: 1 [11520/45000 (26%)]	Loss: 0.163631
Train Epoch: 1 [12160/45000 (27%)]	Loss: 0.106323
Train Epoch: 1 [12800/45000